In [1]:
from os.path import join   
                           
import pandas as pd                                  
import dask.dataframe as dd
import glob

In [2]:
#b = db.read_text('../../data/normalization/raw_texts/ekonomi/*.txt',encoding='cp1254')
#df = dd.read_csv('../../data/normalization/raw_texts/ekonomi/*.txt',encoding='cp1254',sep='\n',header=None,assume_missing=True)
#print(b.compute())
#b.
#files = glob.glob('../../data/normalization/raw_texts/ekonomi/*.txt')
def read_file(path):
    files = glob.glob(path)
    texts=[]
    listToStr=''
    for file in files:
       # open the file and then call .read() to get the text
       with open(file,"r",encoding='cp1254') as f:
          text = f.read()
          texts.append(text)
    listToStr =' '.join(map(str, texts))
    return listToStr

In [3]:
#df = pd.DataFrame(texts, columns=["text"])
relative_path = '../../data/normalization/raw_texts';
economy_category = '/ekonomi'
magazine_category = '/magazin'
medical_category = '/saglik'
sport_category = '/spor'

economy_path = relative_path + economy_category + '/*.txt'
magazine_path = relative_path + magazine_category + '/*.txt'
medical_path = relative_path + medical_category + '/*.txt'
sport_path = relative_path + sport_category + '/*.txt'
stop_words_path = relative_path + '/stopwords.txt'
economy_str = read_file(economy_path)
magazine_str = read_file(magazine_path)
medical_str = read_file(medical_path)
sport_str = read_file(sport_path)
stop_words_str = read_file(stop_words_path)


In [5]:
    from jpype import JClass, JString, getDefaultJVMPath, shutdownJVM, startJVM   
    #shutdownJVM()
    ZEMBEREK_PATH: str = join('..', '..', 'bin', 'zemberek-full.jar')              
    startJVM(                                                                      
        getDefaultJVMPath(),                                                       
        '-ea',                                                                     
        f'-Djava.class.path={ZEMBEREK_PATH}',                                      
        convertStrings=False                                                       
    )                                                                           
               

In [6]:
    TurkishMorphology: JClass = JClass('zemberek.morphology.TurkishMorphology')
    WordAnalysis: JClass = JClass('zemberek.morphology.analysis.WordAnalysis')

    morphology: TurkishMorphology = TurkishMorphology.createWithDefaults()

    word: str = 'kutucuğumuz'

    print(f'\nWord: {word}\n\nResults:')

    results: WordAnalysis = morphology.analyze(JString(word))
    
    for result in results:
        print(
            f'{str(result.getStem())}'
            f'\n\tStems ='
            f' {", ".join([str(result) for result in result.getStems()])}'
            f'\n\tLemmas ='
            f' {", ".join([str(result) for result in result.getStems()])}'
        )


Word: kutucuğumuz

Results:
kutu
	Stems = kutu, kutucuğ
	Lemmas = kutu, kutucuğ
kutu
	Stems = kutu, kutucuğ
	Lemmas = kutu, kutucuğ


In [7]:
    TurkishTokenizer: JClass = JClass('zemberek.tokenization.TurkishTokenizer')    
    Token: JClass = JClass('zemberek.tokenization.Token')                          
    tokenizer: TurkishTokenizer = TurkishTokenizer.DEFAULT                         
                                                                                   
                                                                              
    tokenizer: TurkishTokenizer = TurkishTokenizer.builder().ignoreTypes(          
        Token.Type.Punctuation,                                                    
        Token.Type.NewLine,                                                        
        Token.Type.SpaceTab,                                                       
        Token.Type.Number,
        Token.Type.UnknownWord,
        Token.Type.Unknown,
        Token.Type.URL,
        Token.Type.WordAlphanumerical,
        Token.Type.WordWithSymbol,
        Token.Type.Abbreviation,
        Token.Type.AbbreviationWithDots,
        Token.Type.Punctuation,
        Token.Type.PercentNumeral,
        Token.Type.Time,
        Token.Type.Date,
        Token.Type.URL,
        Token.Type.Email,
        Token.Type.HashTag,
        Token.Type.Mention,
        Token.Type.MetaTag,
        Token.Type.Emoji,
        Token.Type.Emoticon
    ).build() 
    
    economy_tokenized_words = list(tokenizer.tokenize(JString(economy_str)))     
    magazine_tokenized_words = list(tokenizer.tokenize(JString(magazine_str)))                            
    medical_tokenized_words = list(tokenizer.tokenize(JString(medical_str)))                            
    sport_tokenized_words = list(tokenizer.tokenize(JString(sport_str)))                            
    stop_tokenized_words = list(tokenizer.tokenize(JString(stop_words_str)))                  
    
    
    #print(len(words))                                                                        
    #for index,token in enumerate(medical_words):                                   
    #    print(index,token)                                                               
                                                                                   


In [8]:
#wordSet = set(words)
#print(wordSet)

In [9]:
    TurkishMorphology: JClass = JClass('zemberek.morphology.TurkishMorphology')
    WordAnalysis: JClass = JClass('zemberek.morphology.analysis.WordAnalysis')

    morphology: TurkishMorphology = TurkishMorphology.createWithDefaults()
    def getWordsStems(tokenized_words):   
        stems = []
        for word in tokenized_words:
                results: WordAnalysis = morphology.analyze(JString(word.content))
                for result in results:
                    stems.append(str(result.getLemmas()[0]))
        return stems

In [10]:
    def removeStopWords(orginal_words,stop_words):
        words= []
        for word in orginal_words:
            if (word not in stop_words):
                words.append(word)
        return words

In [11]:
   def mapToStringList(stop_words_str):
        words=[]
        for word in stop_words_str:
            words.append(str(word.content))
        return words

In [12]:
#economy_words = getWordsStems(economy_tokenized_words)
#print(economy_words)

In [13]:
    economy_words = getWordsStems(economy_tokenized_words)
    magazine_words = getWordsStems(magazine_tokenized_words)                             
    medical_words = getWordsStems(medical_tokenized_words)                             
    sport_words = getWordsStems(sport_tokenized_words) 
   
    economy_words = removeStopWords(economy_words, mapToStringList(stop_tokenized_words))
    magazine_words = removeStopWords(magazine_words, mapToStringList(stop_tokenized_words))
    medical_words = removeStopWords(medical_words, mapToStringList(stop_tokenized_words))
    sport_words = removeStopWords(sport_words, mapToStringList(stop_tokenized_words))
    corpus_words = economy_words +magazine_words + medical_words +sport_words
    corpus = set(economy_words +magazine_words + medical_words +sport_words)
    #stop_tokenized_words = getWordsStems(economy_tokenized_words) 

In [14]:
#print(economy_words)
print(len(economy_words))
print(len(magazine_words))
print(len(medical_words))
print(len(sport_words))
print(len(corpus))
print(len(corpus_words))
wordDictEconomy = dict.fromkeys(corpus, 0) 
wordDictMagazine = dict.fromkeys(corpus, 0) 
wordDictMedical = dict.fromkeys(corpus, 0) 
wordDictSport = dict.fromkeys(corpus, 0)
wordDictCorpus = dict.fromkeys(corpus, 0)
#print(wordDictEconomy)

125255
59338
93668
100582
10198
378843


In [15]:
for word in economy_words:
    wordDictEconomy[word]+=1
    
for word in magazine_words:
    wordDictMagazine[word]+=1

for word in medical_words:
    wordDictMedical[word]+=1

for word in sport_words:
    wordDictSport[word]+=1
    
for word in corpus_words:
    wordDictCorpus[word]+=1

In [17]:
import pandas as pd
#pd.DataFrame([wordDictA, wordDictB, wordDictC ])
pd.DataFrame([wordDictEconomy,wordDictMagazine,wordDictMedical,wordDictSport,wordDictCorpus ])

,panionios,ayman,bence,orda,görünüş,korun,nakit,stockton,enfraruj,hersek,...,enerjik,arazi,göre,yetki,bosna,cenk,şarap,döşeme,reasürans,sendika
0,0,0,2,1,0,0,11,0,0,0,...,0,2,224,177,1,2,36,0,5,6
1,0,0,9,0,0,0,0,0,0,0,...,0,0,43,4,0,14,6,0,0,0
2,0,0,0,0,0,0,0,0,2,0,...,2,1,103,43,0,2,0,0,0,0
3,1,2,12,0,1,1,1,1,0,1,...,0,3,37,38,2,2,0,2,0,0
4,1,2,23,1,1,1,12,1,2,1,...,2,6,407,262,3,20,42,2,5,6


In [18]:
import math

In [19]:
def combination(m,n):
    return math.factorial(m)//(math.factorial(n)*math.factorial(m-n))

In [20]:

def computeMeaning(wordDictX,wordDictC,bowX,bowC):
    meaningDict = {}
    L = len(bowC)
    B = len(bowX)
    for word, count in wordDictX.items():
        if count == 0:
            meaningDict[word] = (count-1)*math.log10(L/B)
        else:
            meaningDict[word] = (-1/count)*(math.log10(combination(wordDictC[word], count)))- ((count-1)*math.log10(L/B))
    return meaningDict

In [21]:
#meaningBowA = computeMeaning(wordDictA, wordDictC,bowA,bowC)
#meaningBowB = computeMeaning(wordDictB, wordDictC,bowB,bowC)

meaningEconomy = computeMeaning(wordDictEconomy, wordDictCorpus,economy_words,corpus)
meaningMagazine = computeMeaning(wordDictMagazine, wordDictCorpus,magazine_words,corpus)
meaningMedical = computeMeaning(wordDictMedical, wordDictCorpus,medical_words,corpus)
meaningSport = computeMeaning(wordDictSport, wordDictCorpus,sport_words,corpus)
##meaningBowB = computeMeaning(wordDictB, wordDictC,bowB,bowC)


In [22]:
#pd.DataFrame([meaningBowA, meaningBowB])
pd.DataFrame([meaningEconomy, meaningMagazine,meaningMedical,meaningSport])


,panionios,ayman,bence,orda,görünüş,korun,nakit,stockton,enfraruj,hersek,...,enerjik,arazi,göre,yetki,bosna,cenk,şarap,döşeme,reasürans,sendika
0,1.089280,1.089280,-0.112280,0.000000,1.089280,1.089280,10.794693,1.089280,1.089280,1.089280,...,1.089280,0.501234,242.372758,191.315416,-0.477121,-0.050097,37.938141,1.089280,4.357120,5.446400
1,0.764818,0.764818,5.461618,0.764818,0.764818,0.764818,0.764818,0.764818,0.764818,0.764818,...,0.764818,0.764818,30.763458,0.223703,0.764818,9.614891,2.704121,0.764818,0.764818,0.764818
2,0.963076,0.963076,0.963076,0.963076,0.963076,0.963076,0.963076,0.963076,0.963076,0.963076,...,0.963076,-0.778151,97.276043,39.295231,0.963076,-0.176301,0.963076,0.963076,0.963076,0.963076
3,0.000000,0.994005,10.423141,0.994005,0.000000,0.000000,-1.079181,0.000000,0.994005,0.000000,...,0.994005,1.554334,34.360315,35.568940,0.755445,-0.145372,0.994005,0.994005,0.994005,0.994005


In [23]:
pd.DataFrame([meaningEconomy, meaningMagazine,meaningMedical,meaningSport]).mean(axis=0).sort_values()

mira        -0.458023
koşar       -0.458023
şeffaf      -0.408458
yapın       -0.408458
aydan       -0.408458
             ...     
et         768.565882
çok        833.996206
al         835.742561
ol        1619.926346
iç        2021.616919
Length: 10198, dtype: float64

In [24]:
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

In [31]:
tfEconomy = computeTF(wordDictEconomy, economy_words)
tfMagazine = computeTF(wordDictMagazine, magazine_words)
tfMedical = computeTF(wordDictMedical, medical_words)
tfSport = computeTF(wordDictSport, sport_words)

In [32]:
def computeIDF(docList):
    import math
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
        
    return idfDict

In [33]:
idfs = computeIDF([wordDictEconomy, wordDictMagazine,wordDictMedical,wordDictSport])


In [35]:
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [36]:
tfidfEconomy = computeTFIDF(tfEconomy, idfs)
tfidfMagazine = computeTFIDF(tfMagazine, idfs)
tfidfMedical = computeTFIDF(tfMedical, idfs)
tfidfSport = computeTFIDF(tfSport, idfs)

In [37]:
import pandas as pd
pd.DataFrame([tfidfEconomy, tfidfMagazine,tfidfMedical,tfidfSport])

,panionios,ayman,bence,orda,görünüş,korun,nakit,stockton,enfraruj,hersek,...,enerjik,arazi,göre,yetki,bosna,cenk,şarap,döşeme,reasürans,sendika
0,0.000000,0.000000,0.000002,0.000005,0.000000,0.000000,0.000026,0.000000,0.000000,0.000000,...,0.000000,0.000002,0.0,0.0,0.000002,0.0,0.000087,0.000000,0.000024,0.000029
1,0.000000,0.000000,0.000019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000030,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000013,0.000000,...,0.000013,0.000001,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
3,0.000006,0.000012,0.000015,0.000000,0.000006,0.000006,0.000003,0.000006,0.000000,0.000006,...,0.000000,0.000004,0.0,0.0,0.000006,0.0,0.000000,0.000012,0.000000,0.000000


In [ ]:
shutdownJVM()                                                                  